In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, zipfile, pickle
from tqdm import tqdm
from PIL import Image
from transformers import ViTModel
import torch

# ======= Set current ZIP filename ======= GET THE FILE Query
zip_path = f'/content/drive/MyDrive/[EVENTA 2025] Event-Enriched Image Captioning/Release/Track 1 - Private Set/images.zip'
extract_path = f'/content/r100'

# ======= Extract ZIP =======
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
import os, zipfile, pickle
from tqdm import tqdm
from PIL import Image
from transformers import ViTModel
import torch

# ======= Set current ZIP filename ======= GET THE FILE R100
zip_path = f'/content/drive/MyDrive/[EVENTA 2025] Event-Enriched Image Captioning/Database/r100_image_1.zip'
extract_path = f'/content/r100'

# ======= Extract ZIP =======
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
from glob import glob
image_paths = glob(os.path.join(extract_path, '**', '*.jpg'), recursive=True) + \
              glob(os.path.join(extract_path, '**', '*.png'), recursive=True)

print(f"Total: {len(image_paths)} images")



Total: 64299 images
/content/r100/7da06903bdfb247d.jpg


In [ ]:
import torch
import os
from PIL import Image
from tqdm import tqdm
from transformers import AutoModel, AutoImageProcessor
from torchvision import transforms

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = AutoModel.from_pretrained('facebook/dinov2-with-registers-giant')
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-with-registers-giant')
model.eval()

cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Dinov2WithRegistersModel(
  (embeddings): Dinov2WithRegistersEmbeddings(
    (patch_embeddings): Dinov2WithRegistersPatchEmbeddings(
      (projection): Conv2d(3, 1536, kernel_size=(14, 14), stride=(14, 14))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Dinov2WithRegistersEncoder(
    (layer): ModuleList(
      (0-39): 40 x Dinov2WithRegistersLayer(
        (norm1): LayerNorm((1536,), eps=1e-06, elementwise_affine=True)
        (attention): Dinov2WithRegistersAttention(
          (attention): Dinov2WithRegistersSelfAttention(
            (query): Linear(in_features=1536, out_features=1536, bias=True)
            (key): Linear(in_features=1536, out_features=1536, bias=True)
            (value): Linear(in_features=1536, out_features=1536, bias=True)
          )
          (output): Dinov2WithRegistersSelfOutput(
            (dense): Linear(in_features=1536, out_features=1536, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
      

In [ ]:
import os, pickle, json
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn.functional as F

# ======= Setup =======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# ======= Parameters =======
batch_size = 64
image_root = "/content/r100"
retrieve_path = "/content/drive/MyDrive/[EVENTA 2025] Event-Enriched Image Captioning/Dinov2/Giant/giant_retrieval_results.json"
output_rerank_json = "/content/drive/MyDrive/[EVENTA 2025] Event-Enriched Image Captioning/Dinov2/patch_rr_retrieve_top100.json"
tok_k = 100

# ======= Load Retrieve File =======
with open(retrieve_path, 'r') as f:
    retrieve_dict = json.load(f)

reranked_results = {}

# ======= Loop over all queries =======
for query_id, data in tqdm(retrieve_dict.items(), desc="Reranking queries"):
    top_ids = data["retrieved_image_ids"][:tok_k]

    def extract_features(img_id, subfolder=""):
        if subfolder:
            img_path = os.path.join(image_root, subfolder, img_id)
        else:
            img_path = os.path.join(image_root, img_id)

        if not os.path.exists(img_path):
            print(f"[Not Found] {img_path}")
            return None

        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"[Image Load Error] {img_path}: {e}")
            return None

        try:
            inputs = processor(images=image, return_tensors="pt").to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                patch_tokens = outputs.last_hidden_state[:, 1:, :]  # exclude CLS
                return patch_tokens.squeeze(0).cpu()  # [N, D]
        except Exception as e:
            print(f"[Model Error] {img_path}: {e}")
            return None

    # Clear all embeddings
    torch.cuda.empty_cache()

    query_feat = extract_features(query_id, subfolder="query")
    if query_feat is None:
        print(f"[Skip] Query {query_id} failed to extract.")
        continue

    scores = []
    valid_ids = []

    # --- Step 2: Extract and compare top-k ---
    for img_id in top_ids:
        db_feat = extract_features(img_id)
        if db_feat is None:
            continue

        # Mutual Nearest Neighbor similarity
        # Normalize patches
        qf = F.normalize(query_feat, dim=1)
        df = F.normalize(db_feat, dim=1)

        # Compute cosine sim matrix: [Nq, Nd]
        sim = torch.mm(qf, df.T)

        # Top-1 sim from each side
        q2d_max = sim.max(dim=1)[0].mean().item()  # mean over query patches
        d2q_max = sim.max(dim=0)[0].mean().item()  # mean over db patches

        mnp_score = (q2d_max + d2q_max) / 2
        scores.append(mnp_score)
        valid_ids.append(img_id)

    # --- Step 3: Sort by symmetric similarity score ---
    sorted_pairs = sorted(zip(valid_ids, scores), key=lambda x: -x[1])
    sorted_ids = [i for i, _ in sorted_pairs]
    sorted_scores = [s for _, s in sorted_pairs]

    # Keep the remaining original ids (after top-k)
    remaining_ids = [img_id for img_id in data["retrieved_image_ids"] if img_id not in top_ids]

    # Final merged list: reranked top-k + the rest
    final_ids = sorted_ids + remaining_ids

    # Optional: For compatibility, fill the score list with 0.0 for the remaining
    final_scores = sorted_scores + [0.0] * len(remaining_ids)

    reranked_results[query_id] = {
        "retrieved_image_ids": final_ids,
        "similarity_scores": final_scores
    }

# ======= Save reranked results =======
with open(output_rerank_json, "w") as f:
    json.dump(reranked_results, f, indent=2)

print(f"[✅ Done] Reranked results saved to {output_rerank_json}")

Reranking queries:   0%|          | 4/2000 [01:01<8:04:43, 14.57s/it]